In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import datetime
import time
import random
import numpy
from keras.models import load_model

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    var instruction
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '0px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.setAttribute("align", "center");
      modelOut.innerHTML = "<h1 style='font-size: 20px'>Salut ! Je suis le photomaton de Centrale IA !</h1>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instructions = document.createElement('div');
      instructions.setAttribute("align", "center");
      instruction = document.createElement('span');
      instruction.innerText = 'No data';
      instructions.appendChild(instruction);
      div.appendChild(instructions);

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      return stream;
    }
    async function stream_frame(drunks,label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
      if (drunks != "") {
        instruction.innerHTML = drunks;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(drunks, label, bbox):
  data = eval_js('stream_frame("{}","{}", "{}")'.format(drunks,label, bbox))
  return data

# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

# start streaming video from webcam
video_stream()
# label for video
# initialze bounding box to empty
bbox = ''
count = 0 
labels = ['Totalement sobre','Jsuis bien la','A 2 doigts de rappeller mon ex','Niveau Joseph']
colors = [(0,255,0),(85,170,0),(170,85,0),(255,0,0)]
model = load_model('/content/drive/MyDrive/CentraleIADataset/centralia.h5')
instruction = ''
label_html = ''
last_one = datetime.datetime.now()
while True:
    js_reply = video_frame(instruction,label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    out_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) 
    # get face region coordinates
    faces = face_cascade.detectMultiScale(gray)
    label_html = str(datetime.datetime.now().time())
    # get face bounding box for overlay
    drunks = [0,0]
    last_photo = -1
    for (x,y,w,h) in faces:
      factor = 7
      xi,yi,wi,hi = round(x/factor),round(y/factor),round(w*factor),round(h*factor)
      crop = out_img[yi:yi+hi,xi:xi+wi]
      crop = cv2.resize(crop,(128,128))
      crop = numpy.reshape(crop,[1,128,128,3])/255.0
      mask_result = model.predict(crop)
      #print(mask_result[0][0])
      mask_flag = mask_result.argmax()
      if mask_result[0][0] <= 0.88:
        label = labels[0]
        color = colors[0]
        drunks[0]+=1
      elif mask_result[0][0] > 0.88 and mask_result[0][0] <= 0.93:
        label = labels[1]
        color = colors[1]
        drunks[0]+=1
      elif mask_result[0][0] > 0.93 and mask_result[0][0] <= 0.97:
        label = labels[2]
        color = colors[2]
        drunks[1]+=1
      elif mask_result[0][0] > 0.97:
        label = labels[3]
        color = colors[3]
        drunks[1]+=1
      cv2.putText(bbox_array,label,(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),color,1)
    if drunks[1] > drunks[0] and sum(drunks)>0:
      instruction = "<h1 style='font-size: 20px'>Viens m'essayer et je te dirais à quel point tu es bourré!</h1><h1 style='font-size: 20px'>(Detecté)</h1>"
      if (datetime.datetime.now() - last_one).total_seconds()> 10.0 :
        last_one = datetime.datetime.now()
        instruction = "<h1 style='font-size: 40px'>Photo!</h1>"
        cv2.imwrite('/content/drive/MyDrive/CentraleIADataset/NDet/'+str(datetime.datetime.now())+'.png',img)
        cv2.imwrite('/content/drive/MyDrive/CentraleIADataset/Det/'+str(datetime.datetime.now())+'.png',bbox_array)
    else:
      instruction = "<h1 style='font-size: 20px'>Viens m'essayer et je te dirais à quel point tu es bourré</h1><h1 style='font-size: 20px'>(Non Beurré)</h1>"
      # cv2.putText(bbox_array,"DRUNKS!",(len(img[0])//2-200, 50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
    #   cv2.imwrite('test', img);
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

<IPython.core.display.Javascript object>